## 4. Modeling

In this notebook, I will train machine learning models to power the recommender system.  
I will be using the [scikit-surprise library](https://surprise.readthedocs.io/en/stable/) to build the recommender system.

In [1]:
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from surprise.model_selection import KFold
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from random import sample, choice

In [2]:
reading_no_zero_df = pd.read_csv('../data/reading_no_zero.csv')

### Machine Learning models in scikit-surprise

I will be trying out all of the [scikit-surprise models](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html) except SVDpp. During my literature review of other recommender system projects and trying out SVDpp myself, I have concluded that it has an extremely long fit time with similar results to the other models.  

The explanation for each of the models used is taken from the [scikit-surprise documentation](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html).

- Normal Predictor: Algorithm predicting a random rating based on the distribution of the training set, which is assumed to be normal.
- Baseline Only: Algorithm predicting the baseline estimate for given user and item.
- KNN Basic: A basic collaborative filtering algorithm.
- KNN with Means: A basic collaborative filtering algorithm, taking into account the mean ratings of each user.
- KNN with Z-scores: A basic collaborative filtering algorithm, taking into account the z-score normalization of each user.
- KNN Baseline: A basic collaborative filtering algorithm taking into account a baseline rating.
- SVD: The famous SVD algorithm, as popularized by Simon Funk during the Netflix Prize.When baselines are not used, this is equivalent to Probabilistic Matrix Factorization [salakhutdinov2008a]
- Non-negative Matrix Factorization: A collaborative filtering algorithm based on Non-negative Matrix Factorization.
- Slope One: A simple yet accurate collaborative filtering algorithm.
- Co-clustering: A collaborative filtering algorithm based on co-clustering.

In [70]:
algos = {'SVD':SVD(), 'Slope One':SlopeOne(), 'Non-negative Matrix Factorization':NMF(), 'Normal Predictor':NormalPredictor(), 'KNN Baseline':KNNBaseline(), 'KNN Basic':KNNBasic(), 'KNN with Means':KNNWithMeans(), 'KNN with Z-Score':KNNWithZScore(), 'Basline Only':BaselineOnly(), 'Co-clustering':CoClustering()}
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(reading_no_zero_df[['user', 'item', 'rating']], reader)

In [71]:
overall_results = []
for algo_name, algo in tqdm(algos.items()): 
    algo_results = []
    results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
    algo_results.append(algo_name)
    algo_results.append(results['test_rmse'].mean())
    overall_results.append(algo_results)

 40%|█████████████████████████████████▏                                                 | 4/10 [02:31<03:15, 32.66s/it]

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


 50%|█████████████████████████████████████████▌                                         | 5/10 [04:58<06:10, 74.01s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


 60%|█████████████████████████████████████████████████▊                                 | 6/10 [07:15<06:22, 95.59s/it]

...

 70%|█████████████████████████████████████████████████████████▍                        | 7/10 [35:19<30:44, 614.77s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


 80%|█████████████████████████████████████████████████████████████████▌                | 8/10 [37:45<15:31, 465.51s/it]

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [38:25<00:00, 230.54s/it]


In [72]:
overall_results_df = pd.DataFrame(overall_results, columns = ['Algorithm Name', 'Mean CV RMSE Score'])
overall_results_df.sort_values('Mean CV RMSE Score')

,Algorithm Name,Mean CV RMSE Score
1,Slope One,1.201943
4,KNN Baseline,1.203653
0,SVD,1.213756
8,Basline Only,1.215756
7,KNN with Z-Score,1.238657
6,KNN with Means,1.242536
9,Co-clustering,1.259351
5,KNN Basic,1.336306
3,Normal Predictor,2.144525
2,Non-negative Matrix Factorization,2.245515


**Data Shown:** The RMSE scores shown are the average over 5 cross-validation folds. The better performing models have an error in the range of 1.2+. For a rating system out of 10, this is around 12% error.

**Insights:** By default in scikit-surprise, the similarity between users is calculated via their [mean squared difference](https://surprise.readthedocs.io/en/stable/similarities.html?highlight=msd#surprise.similarities.msd).

![msd](../images/msd.JPG)

Notation explanation from [scikit-surprise](https://surprise.readthedocs.io/en/stable/notation_standards.html#notation-standards)
- U : the set of all users. u and v denotes users.
- I : the set of all items. i and j denotes items.
- rui : the true rating of user u for item i.

In [73]:
sim_options = {'name':'cosine'}
algos = {'KNN Baseline':KNNBaseline(sim_options = sim_options), 'KNN Basic':KNNBasic(sim_options = sim_options), 'KNN with Means':KNNWithMeans(sim_options = sim_options), 'KNN with Z-Score':KNNWithZScore(sim_options = sim_options)}
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(reading_no_zero_df[['user', 'item', 'rating']], reader)

overall_results_cosine = []
for algo_name, algo in tqdm(algos.items()): 
    algo_results = []
    results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
    algo_results.append(algo_name)
    algo_results.append(results['test_rmse'].mean())
    overall_results_cosine.append(algo_results)
    
overall_results_cosine_df = pd.DataFrame(overall_results_cosine, columns = ['Algorithm Name', 'Mean CV RMSE Score (cosine)'])
overall_results_cosine_df.sort_values('Mean CV RMSE Score (cosine)')

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


 25%|████████████████████▊                                                              | 1/4 [04:17<12:51, 257.22s/it]

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


 50%|█████████████████████████████████████████▌                                         | 2/4 [08:20<08:18, 249.29s/it]

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [12:27<04:07, 247.83s/it]

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [16:40<00:00, 250.04s/it]


,Algorithm Name,Mean CV RMSE Score (cosine)
0,KNN Baseline,1.224187
3,KNN with Z-Score,1.248752
2,KNN with Means,1.253170
1,KNN Basic,1.450262


**Data Shown:** Try out [cosine similarity](https://surprise.readthedocs.io/en/stable/similarities.html?highlight=msd#surprise.similarities.cosine) instead.

![cosine](../images/cosine.JPG)

In [74]:
combined_results_df = pd.merge(left = overall_results_cosine_df, right = overall_results_df, how = 'right', on = 'Algorithm Name')
combined_results_df.sort_values('Mean CV RMSE Score')

,Algorithm Name,Mean CV RMSE Score (cosine),Mean CV RMSE Score
1,Slope One,NaN,1.201943
4,KNN Baseline,1.224187,1.203653
0,SVD,NaN,1.213756
8,Basline Only,NaN,1.215756
7,KNN with Z-Score,1.248752,1.238657
6,KNN with Means,1.253170,1.242536
9,Co-clustering,NaN,1.259351
5,KNN Basic,1.450262,1.336306
3,Normal Predictor,NaN,2.144525
2,Non-negative Matrix Factorization,NaN,2.245515


**Data Shown:** All average RMSE score for 5 CV folds.

**Insights:** Cosine similarity version performed worse than the default msd similarity.

----
### Precision@k and Recall@k

Rather than using RMSE to gauge the accuracy of a model, Precision@k and Recall@k are more commonly used to measure the performance of a recommender system.

![cosine](../images/precision_recall.png)

k in this case refers to the number of recommendations. Relevance is defined as recommendations with a score equal to or higher than a defined threshold.   
For example, Precision@10 with threshold 7 refers to the number of recommendations that the user rated >= 7, divided by the 10 recommendations that the model makes.

These metrics prioritize the quality of the top k recommendations over the average quality of all recommendations like RMSE. This makes sense practically as well as the user will only care about the top k recommendations being shown, rather than every single item's rating.  

### Measurement metrics for this project
For this project, I am opting for top 10 recommendations, i.e. k = 10 and a threshold rating of 7/10.

### Baseline Model

For the baseline model, it will randomly pick 10 manga titles to recommend to each user. The precision@k and recall@k scores are then calculated using these 10 random recommendations.

In [36]:
list_of_titles = reading_no_zero_df['item'].unique().tolist()
list_of_users = reading_no_zero_df['user'].unique().tolist()
k = 10

# Precision@k calculation using random 1000 experiments
precision = []
recall = []

for i in tqdm(range(reading_no_zero_df['user'].nunique())):
    random_user = choice(list_of_users)
    random_user_titles = reading_no_zero_df[(reading_no_zero_df['user']==random_user) & (reading_no_zero_df['rating']>=7)]['item'].unique().tolist()
    total_relevant_titles = reading_no_zero_df[(reading_no_zero_df['user']==random_user) & (reading_no_zero_df['rating']>=7)]['item'].nunique()
    
    # Since all titles can be selected, we select a random k titles
    random_10_titles = sample(list_of_titles, k)    
    num_correct_titles = []
    for title in random_10_titles:
        if title in random_user_titles:
            num_correct_titles.append(title)

    if total_relevant_titles == 0:
        recall.append(0)
    else:
        recall.append(len(num_correct_titles)/total_relevant_titles)
    
    precision.append(len(num_correct_titles)/10)
    
    list_of_users.remove(random_user)

precision_at_k = sum(precision)/len(precision)
recall_at_k = sum(recall)/len(recall)

print(f'Average precision@k over 1,000 experiments is {precision_at_k}')
print(f'Average recall@k over 1,000 experiments is {recall_at_k}')

100%|████████████████████████████████████████████████████████████████████████████| 14138/14138 [06:02<00:00, 38.96it/s]

Average precision@k over 1,000 experiments is 0.007794596123921236
Average recall@k over 1,000 experiments is 0.00338124770300819


In [39]:
baseline_results = pd.DataFrame(columns = ['Algorithm Name', 'Precision@k', 'Recall@k'])
baseline_results.loc[0] = ['Baseline (Random Recommendations)',precision_at_k, recall_at_k]
baseline_results

,Algorithm Name,Precision@k,Recall@k
0,Baseline (Random Recommendations),0.007795,0.003381


In [21]:
# from surprise FAQ https://surprise.readthedocs.io/en/stable/FAQ.html
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [82]:
#testing out the calculation of mean precision@k and recall@k scores
kf = KFold(n_splits=5)
precision_list = []
recall_list = []
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=7)
    precision_list.append(sum(prec for prec in precisions.values()) / len(precisions))
    recall_list.append(sum(rec for rec in recalls.values()) / len(recalls))
    # Precision and recall can then be averaged over all users
print(f'Precision Scores are: {precision_list}')
print(f'Average Precision Score is: {sum(precision_list)/len(precision_list)}')
print(f'Recall Scores are: {recall_list}')
print(f'Average Recall Score is: {sum(recall_list)/len(recall_list)}')

<ipython-input-82-67d433ec0ae0>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo.fit(trainset)


Precision Scores are: [0.8980473943124514, 0.89832656888971, 0.8968780208383775, 0.8986979235931962, 0.8984706136680022]
Average Precision Score is: 0.8980841042603475
Recall Scores are: [0.8142019043070349, 0.8164423999987825, 0.8140727508183734, 0.8162317487504723, 0.8154849222986732]
Average Recall Score is: 0.8152867452346673


In [23]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(reading_no_zero_df[['user', 'item', 'rating']], reader)

In [91]:
algos = {'SVD':SVD(), 'Slope One':SlopeOne(), 'Non-negative Matrix Factorization':NMF(), 'Normal Predictor':NormalPredictor(), 'KNN Baseline':KNNBaseline(), 'KNN Basic':KNNBasic(), 'KNN with Means':KNNWithMeans(), 'KNN with Z-Score':KNNWithZScore(), 'Basline Only':BaselineOnly(), 'Co-clustering':CoClustering()}
kf = KFold(n_splits=5, random_state = 42)
all_precision_recall = []
for algo_name, algo in tqdm(algos.items()):
    algo_precision_recall_list = []
    precision_list = []
    recall_list = []
    
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=7)
        precision_list.append(sum(prec for prec in precisions.values()) / len(precisions))
        recall_list.append(sum(rec for rec in recalls.values()) / len(recalls))
    
    precision_average = sum(precision_list)/len(precision_list)
    recall_average = sum(recall_list)/len(recall_list)
    algo_precision_recall_list.append(algo_name)
    algo_precision_recall_list.append(precision_average)
    algo_precision_recall_list.append(recall_average)
    all_precision_recall.append(algo_precision_recall_list)

all_precision_recall_df = pd.DataFrame(all_precision_recall, columns = ['Algorithm Name', 'Average Precision@k Score', 'Average Recall@k Score'])
all_precision_recall_df.sort_values('Average Precision@k Score', ascending = False)

 10%|████████▎                                                                          | 1/10 [01:02<09:19, 62.14s/it]<ipython-input-91-7abf4afd8948>:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo.fit(trainset)
 40%|█████████████████████████████████▏                                                 | 4/10 [02:46<03:36, 36.02s/it]

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


 50%|█████████████████████████████████████████▌                                         | 5/10 [05:22<06:35, 79.19s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


 60%|█████████████████████████████████████████████████▏                                | 6/10 [07:47<06:46, 101.69s/it]

...

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


 80%|█████████████████████████████████████████████████████████████████▌                | 8/10 [12:52<04:18, 129.22s/it]

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [13:39<00:00, 81.92s/it]


,Algorithm Name,Average Precision@k Score,Average Recall@k Score
8,Basline Only,0.909319,0.862302
0,SVD,0.909265,0.846149
4,KNN Baseline,0.908539,0.850089
5,KNN Basic,0.901041,0.854036
1,Slope One,0.898290,0.815146
7,KNN with Z-Score,0.896743,0.826443
6,KNN with Means,0.892933,0.823171
9,Co-clustering,0.892634,0.820007
3,Normal Predictor,0.803468,0.648952
2,Non-negative Matrix Factorization,0.406589,0.206171


In [19]:
#defining a function to calculate precision@k and recall@k
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(reading_no_zero_df[['user', 'item', 'rating']], reader)
sim_options = {'name':'cosine'}
def calculate_precision_recall(k, threshold, splits):
    algos = {'SVD':SVD(), 'Slope One':SlopeOne(), 'Non-negative Matrix Factorization':NMF(), 'Normal Predictor':NormalPredictor(), 'KNN Baseline':KNNBaseline(), 'KNN Basic':KNNBasic(), 'KNN with Means':KNNWithMeans(), 'KNN with Z-Score':KNNWithZScore(), 'Basline Only':BaselineOnly(), 'Co-clustering':CoClustering(), 'KNN Baseline_Cosine':KNNBaseline(sim_options = sim_options), 'KNN Basic_Cosine':KNNBasic(sim_options = sim_options), 'KNN with Means_Cosine':KNNWithMeans(sim_options = sim_options), 'KNN with Z-Score_Cosine':KNNWithZScore(sim_options = sim_options)}
    kf = KFold(n_splits=splits, random_state = 42)
    all_precision_recall = []
    for algo_name, algo in tqdm(algos.items()):
        algo_precision_recall_list = []
        precision_list = []
        recall_list = []

        for trainset, testset in kf.split(data):
            algo.fit(trainset)
            predictions = algo.test(testset)
            precisions, recalls = precision_recall_at_k(predictions, k, threshold)
            precision_list.append(sum(prec for prec in precisions.values()) / len(precisions))
            recall_list.append(sum(rec for rec in recalls.values()) / len(recalls))

        precision_average = sum(precision_list)/len(precision_list)
        recall_average = sum(recall_list)/len(recall_list)
        algo_precision_recall_list.append(algo_name)
        algo_precision_recall_list.append(precision_average)
        algo_precision_recall_list.append(recall_average)
        all_precision_recall.append(algo_precision_recall_list)

    all_precision_recall_df = pd.DataFrame(all_precision_recall, columns = ['Algorithm Name', 'Average Precision@k Score', 'Average Recall@k Score'])
    return all_precision_recall_df.sort_values('Average Recall@k Score', ascending = False)



In [93]:
calculate_precision_recall(k = 10, threshold = 7, splits = 5)

  7%|█████▉                                                                             | 1/14 [01:00<13:09, 60.76s/it]<ipython-input-92-98dc9d4b62ba>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo.fit(trainset)
 29%|███████████████████████▋                                                           | 4/14 [02:45<06:00, 36.01s/it]

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


 36%|█████████████████████████████▋                                                     | 5/14 [05:19<11:46, 78.48s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


 43%|███████████████████████████████████▏                                              | 6/14 [07:42<13:23, 100.40s/it]

...

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


 93%|███████████████████████████████████████████████████████████████████████████▏     | 13/14 [25:53<03:10, 190.96s/it]

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [30:13<00:00, 129.53s/it]


,Algorithm Name,Average Precision@k Score,Average Recall@k Score
8,Basline Only,0.909319,0.862302
5,KNN Basic,0.901041,0.854036
10,KNN Baseline_Cosine,0.905999,0.852399
4,KNN Baseline,0.908539,0.850089
11,KNN Basic_Cosine,0.895532,0.848228
0,SVD,0.909108,0.845995
13,KNN with Z-Score_Cosine,0.894589,0.827692
7,KNN with Z-Score,0.896743,0.826443
12,KNN with Means_Cosine,0.890518,0.823206
6,KNN with Means,0.892933,0.823171


**Observation:** Function is working as intended as the results are the same as above.

In [94]:
calculate_precision_recall(k = 10, threshold = 8, splits = 5)

  7%|█████▉                                                                             | 1/14 [01:01<13:15, 61.17s/it]<ipython-input-92-98dc9d4b62ba>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo.fit(trainset)
 29%|███████████████████████▋                                                           | 4/14 [02:42<05:49, 34.99s/it]

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


 36%|█████████████████████████████▋                                                     | 5/14 [05:12<11:27, 76.41s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


 43%|███████████████████████████████████▌                                               | 6/14 [07:30<13:01, 97.66s/it]

...

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


 93%|███████████████████████████████████████████████████████████████████████████▏     | 13/14 [42:04<04:54, 294.97s/it]

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [46:15<00:00, 198.24s/it]


,Algorithm Name,Average Precision@k Score,Average Recall@k Score
4,KNN Baseline,0.738545,0.620793
0,SVD,0.734596,0.612717
8,Basline Only,0.735769,0.611244
10,KNN Baseline_Cosine,0.729246,0.608770
5,KNN Basic,0.709247,0.599862
9,Co-clustering,0.682340,0.597335
7,KNN with Z-Score,0.680021,0.596183
13,KNN with Z-Score_Cosine,0.669334,0.587712
6,KNN with Means,0.670122,0.586698
12,KNN with Means_Cosine,0.661233,0.577683


In [95]:
calculate_precision_recall(k = 10, threshold = 9, splits = 5)

  7%|█████▉                                                                             | 1/14 [00:59<12:50, 59.26s/it]<ipython-input-92-98dc9d4b62ba>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo.fit(trainset)
 29%|███████████████████████▋                                                           | 4/14 [02:38<05:42, 34.29s/it]

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


 36%|█████████████████████████████▋                                                     | 5/14 [05:06<11:18, 75.34s/it]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


 43%|███████████████████████████████████▌                                               | 6/14 [07:23<12:49, 96.17s/it]

...

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


 93%|███████████████████████████████████████████████████████████████████████████▏     | 13/14 [27:33<03:33, 213.17s/it]

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [32:16<00:00, 138.29s/it]


,Algorithm Name,Average Precision@k Score,Average Recall@k Score
9,Co-clustering,0.341249,0.276210
6,KNN with Means,0.304150,0.249917
7,KNN with Z-Score,0.300952,0.243792
12,KNN with Means_Cosine,0.294070,0.238788
1,Slope One,0.334473,0.236897
13,KNN with Z-Score_Cosine,0.286146,0.232402
0,SVD,0.303585,0.186566
3,Normal Predictor,0.257094,0.183974
4,KNN Baseline,0.285128,0.172828
10,KNN Baseline_Cosine,0.256682,0.146398


**Data Shown:** 3 sets of results all based on k = 10 but varying thresholds of 7, 8 and 9 respectively. 

**Insights:** As expected, the results deprove as the threshold is continuously increased. Personally, I find 7 out of 10 to be a good enough score for a recommendation. Rather than choosing a model that performs less badly at a higher threshold, I want the best model at recommending a good enough manga title. Hence, I will be using the Baseline Only model that performed the best at threshold = 7. Its recall and precision scores are almost a whole 1% higher than the 2nd best model.

The ML models vastly outperform the baseline random experiments model results.